CSC 820  
Homework 7    
Andrew Dahlstrom  
03/20/2024  



In [13]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import re
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import nltk
from nltk.tokenize import TweetTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
import xgboost
from sklearn.model_selection import RandomizedSearchCV
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [14]:
# Please update the file paths to your current file location for for the dataset. From kaggle you can use
# the provided path below
# /kaggle/input/covid-19-nlp-text-classification/Corona_NLP_train.csv
# Alternately you can download the file into your one drive and unzip it and store
# the unzipped files in the local directory as I did below.
!unzip "/content/drive/MyDrive/kaggle/archive.zip"
file_path_train = "/content/Corona_NLP_train.csv"
file_path_test =  "/content/Corona_NLP_test.csv"

Archive:  /content/drive/MyDrive/kaggle/archive.zip
replace Corona_NLP_test.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace Corona_NLP_train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [15]:
# Load training dataset into a pandas dataframe from csv using the latin1 enconding which is the standard English alphabet
# plus a range of other characters from other European languages, including characters with accents.
train=pd.read_csv(file_path_train,encoding='latin1')
# Show first 5 rows
train.head()

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
0,3799,48751,London,16-03-2020,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,Neutral
1,3800,48752,UK,16-03-2020,advice Talk to your neighbours family to excha...,Positive
2,3801,48753,Vagabonds,16-03-2020,Coronavirus Australia: Woolworths to give elde...,Positive
3,3802,48754,NaN,16-03-2020,My food stock is not the only one which is emp...,Positive
4,3803,48755,NaN,16-03-2020,"Me, ready to go at supermarket during the #COV...",Extremely Negative


In [16]:
# Define a function to drop unused columns from the dataset
def drop(p):
    p.drop(["UserName","ScreenName","Location","TweetAt"],axis=1,inplace=True)

In [17]:
# Drop the unused columns and display first 5 rows
drop(train)
train.head()

,OriginalTweet,Sentiment
0,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,Neutral
1,advice Talk to your neighbours family to excha...,Positive
2,Coronavirus Australia: Woolworths to give elde...,Positive
3,My food stock is not the only one which is emp...,Positive
4,"Me, ready to go at supermarket during the #COV...",Extremely Negative


In [18]:
# Display the number of samples for each label
train["Sentiment"].value_counts()

Positive              11422
Negative               9917
Neutral                7713
Extremely Positive     6624
Extremely Negative     5481
Name: Sentiment, dtype: int64

In [19]:
# Define a function to replace text sentiment labels with numeric labels for easier modeling
def rep(t):
        d={"Sentiment":{'Positive':0,'Negative':1,"Neutral":2,"Extremely Positive":3,"Extremely Negative":4}}
        t.replace(d,inplace=True)

In [20]:
# Apply the function to transform the labels and display the first 5 rows
rep(train)
train.head()

,OriginalTweet,Sentiment
0,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,2
1,advice Talk to your neighbours family to excha...,0
2,Coronavirus Australia: Woolworths to give elde...,0
3,My food stock is not the only one which is emp...,0
4,"Me, ready to go at supermarket during the #COV...",4


In [21]:
# Initialize a tokenizer, lemmatizer and stemmer for text processing
tweettoken = TweetTokenizer(strip_handles=True, reduce_len=True)
lemmatizer=WordNetLemmatizer()
stemmer=PorterStemmer()

In [22]:
# Define preprocess function given a text string input, standardize it and parse it by
# removing non-alphabetic characters, convert letters to lower case, tokenize into
# works, remove stopwords, lammatize remaining words, combine back into a text string and
# and it to the collect array.
collect=[]
def preprocess(t):
    tee=re.sub('[^a-zA-Z]'," ",t)
    tee=tee.lower()
    res=tweettoken.tokenize(tee)
    for i in res:
        if i in stopwords.words('english'):
            res.remove(i)
    rest=[]
    for k in res:
        rest.append(lemmatizer.lemmatize(k))
    ret=" ".join(rest)
    collect.append(ret)

In [25]:
# Apply the preprocess function to each tweet in the dataset
nltk.download('wordnet')
nltk.download('stopwords')
for j in range(41157):
    preprocess(train["OriginalTweet"].iloc[j])

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [26]:
# Display the first five preprocessed tweets
collect[:5]

['menyrbie phil gahan chrisitv http co ifz fan pa http co xx ghgfzcc http co nlzdxno',
 'advice talk your neighbour family exchange phone number create contact list phone number neighbour school employer chemist gp set online shopping account po adequate supply regular med not order',
 'coronavirus australia woolworth give elderly disabled dedicated shopping hour amid covid outbreak http co binca vp p',
 'food stock not only one is empty please panic will enough food everyone not take than you need stay calm stay safe covid france covid covid coronavirus confinement confinementotal confinementgeneral http t co zrlg z j',
 'ready go supermarket covid outbreak because m paranoid because food stock litteraly empty the coronavirus a serious thing please panic cause shortage coronavirusfrance restezchezvous stayathome confinement http t co usmualq n']

In [27]:
# Define a bag of words function to convert a list of text strings into a count of unique
# words up to the 200 most frequently used words
def bow(ll):
    cv=CountVectorizer(max_features=200)
    x=cv.fit_transform(ll).toarray()
    return x

In [42]:
# Apply the bag-of-words model to the preprocessed text data
y=bow(collect)
y[:1]

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0]])

In [43]:
# Display the number of features, unique words
len(y[0][:])

200

In [44]:
# "Tf-idf is one of the best metrics to determine how significant a term is to a text
# in a series or a corpus. tf-idf is a weighting system that assigns a weight to
# each word in a document based on its term frequency (tf) and the reciprocal
# document frequency (tf) (idf). The words with higher scores of weight are deemed
# to be more significant."
# (https://www.geeksforgeeks.org/understanding-tf-idf-term-frequency-inverse-document-frequency/)
# Define a tfidf function to convert text into a TF-IDF model array with max features of 4000
def tfidf(xx):
    cv=TfidfVectorizer(max_features=4000)
    x=cv.fit_transform(xx).toarray()
    return x

In [45]:
# Prepare sentiment labels for model training and testing
values=train["Sentiment"].values
values

array([2, 0, 0, ..., 0, 2, 1])

In [46]:
# Split the dataset into training and test sets for model evaluation
(x_train,x_test,y_train,y_test) = train_test_split(y,values, train_size=0.75, random_state=42)
x_train

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [47]:
# Train a Random Forest classifier,
# evaluate its accuracy on the test set,
# and compute a confusion matrix
rnd_clf=RandomForestClassifier(n_estimators=200,random_state=42)
rnd_clf.fit(x_train,y_train)
rnd_clf.score(x_test,y_test)
y_pred=rnd_clf.predict(x_test)
cm=confusion_matrix(y_test,y_pred)
cm

array([[1358,  592,  501,  338,  109],
       [ 702, 1014,  448,  106,  243],
       [ 477,  438,  913,   46,   45],
       [ 653,  183,  162,  617,   28],
       [ 287,  457,  148,   33,  392]])

In [34]:
# Experiment with different estimators in the Random Forest classifier
a=[400,500,600,700,800,900,1000]
for i in a:
    rnd_clf=RandomForestClassifier(n_estimators=i,random_state=42)
    rnd_clf.fit(x_train,y_train)
    t=rnd_clf.score(x_test,y_test)
    print(t)

0.4152575315840622
0.41438289601554906
0.41438289601554906
0.41243926141885323
0.41564625850340137
0.4182701652089407
0.41778425655976675


In [48]:
# Train a Naive Bayes classifier
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(x_train,y_train)

MultinomialNB()

In [49]:
# Evaluate Naive Bayes classifier accuracy on the test set
clf.score(x_test,y_test)

0.3825072886297376

In [50]:
# Compute the confusion matrix for the Naive Bayes
y_pred = clf.predict(x_test)
cm=confusion_matrix(y_test,y_pred)
cm

array([[1001,  512,  709,  502,  174],
       [ 586,  769,  601,  177,  380],
       [ 406,  325, 1061,   62,   65],
       [ 516,  153,  233,  670,   71],
       [ 240,  384,  200,   58,  435]])

In [51]:
# Apply TF-IDF processor to the text data to create feature vectors
y=tfidf(collect)
# Split the TF-IDF feature vectors and sentiment values into training and test sets
(x_train,x_test,y_train,y_test) = train_test_split(y,values, train_size=0.75, random_state=42)

In [52]:
# Initialize and train a Random Forest classifier with
# 200 estimators on TF-IDF training set and evaluate its accuracy
rnd_clf=RandomForestClassifier(n_estimators=200,max_leaf_nodes=8,random_state=42)
rnd_clf.fit(x_train,y_train)
rnd_clf.score(x_test,y_test)

0.2937803692905734

In [53]:
# Initialize and train a Naive Bayes classifier with
# on TF-IDF training set and evaluate its accuracy
clf = MultinomialNB()
clf.fit(x_train,y_train)
clf.score(x_test,y_test)

0.46763848396501456

In [54]:
# Compute the confusion matrix for the Naive Bayes after TF-IDF processing
y_pred = clf.predict(x_test)
cm=confusion_matrix(y_test,y_pred)
cm

array([[2066,  491,  195,  131,   15],
       [ 912, 1295,  186,   26,   94],
       [ 767,  350,  774,   20,    8],
       [1114,   78,   34,  415,    2],
       [ 214,  807,   32,    2,  262]])

In [ ]:
# The *.score() function is a method for evaluating the performance of a classifier
# or regressor. When applied to a classifier, it calculates the accuracy of the model
# which is (the total # of correct predictions) / (the total # of predictions).

# The difference in the accuracy scores 0.3825 vs 0.4676 corresponds to applying the
# TF-IDF processor to the Naive Bayes model. By examining the two confusion matrices,
# the confusion matrix after the TF-IDF has been applied shows higher values accross the
# diagonal of the matrix indicating a greater number of correct predictions for each class.
# We can therefore see that the model accuracy has improved after TF-IDF.